In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.preprocessing
from sklearn.metrics import r2_score
from pandas import read_csv
from datetime import datetime
from keras.layers import Dense,Dropout,SimpleRNN,LSTM
from keras.models import Sequential

In [ ]:
# load data
dataset = read_csv('Energy.csv')
dataset.head()

In [ ]:
dataset.drop(dataset.index[[0,1,2]], inplace=True)
# manually specify column names
dataset.columns = ['datetime','SIB1001', 'HAR2002', 'TIM2002']
# summarize first 5 rows
dataset.head()

In [ ]:
# iterating the columns to get column names
for col in dataset.columns: 
    print(col)

In [ ]:
#Creating subsets for univariate analysis
SIB1001DF = dataset[['datetime', 'SIB1001']]
HAR2002DF = dataset[['datetime', 'HAR2002']]
TIM2002DF = dataset[['datetime', 'TIM2002']]

In [6]:
# Univariate Analysis for SIB1001

In [ ]:
#dropping rows with missing data
SIB1001DF = SIB1001DF.dropna()
#checking missing data
SIB1001DF.isna().sum()

#column data type formatting
SIB1001DF.datetime = pd.to_datetime(SIB1001DF.datetime)
SIB1001DF.SIB1001 = pd.to_numeric(SIB1001DF.SIB1001)
SIB1001DF.set_index(['datetime'], inplace = True)
SIB1001DF.head()

#visualize data before normalization
SIB1001DF.plot(figsize=(16,4),legend=True)

plt.title('Hourly power consumption data - BEFORE NORMALIZATION')

plt.show()

#normalize data
def normalize_data(SIB1001DF):
    scaler = sklearn.preprocessing.MinMaxScaler()
    SIB1001DF['SIB1001']=scaler.fit_transform(SIB1001DF['SIB1001'].values.reshape(-1,1))
    return SIB1001DF
SIB1001DF_norm = normalize_data(SIB1001DF)
SIB1001DF_norm.shape

#visualize data after normalization
SIB1001DF_norm.plot(figsize=(16,4),legend=True)
plt.title('SIB1001 hourly power consumption data - AFTER NORMALIZATION')
plt.show()

SIB1001DF_norm.shape

#prepare data for training the RNN models
def load_data(stock, seq_len):
    X_train = []
    y_train = []
    for i in range(seq_len, len(stock)):
        X_train.append(stock.iloc[i-seq_len : i, 0])
        y_train.append(stock.iloc[i, 0])
    
    #1 last 3187 days are going to be used in test
    X_test = X_train[21000:]             
    y_test = y_train[21000:]
    
    #2 first 21000 days are going to be used in training
    X_train = X_train[:21000]           
    y_train = y_train[:21000]
    
    #3 convert to numpy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    #4 reshape data to input into RNN models
    X_train = np.reshape(X_train, (21000, seq_len, 1))
    
    X_test = np.reshape(X_test, (X_test.shape[0], seq_len, 1))
    
    return [X_train, y_train, X_test, y_test]

#create train, test data
seq_len = 20 #choose sequence length
X_train, y_train, X_test, y_test = load_data(SIB1001DF, seq_len)
print('X_train.shape = ',X_train.shape)
print('y_train.shape = ', y_train.shape)
print('X_test.shape = ', X_test.shape)
print('y_test.shape = ',y_test.shape)
X_train.shape =  (21000, 20, 1)
y_train.shape =  (21000,)
X_test.shape =  (3797, 20, 1)
y_test.shape =  (3797,)

#build a simple RNN model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=True, input_shape=(X_train.shape[1],1)))
rnn_model.add(Dropout(0.15))
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=True))
rnn_model.add(Dropout(0.15))
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=False))
rnn_model.add(Dropout(0.15))
rnn_model.add(Dense(1))
rnn_model.summary()

#evaluate model performance
rnn_model.compile(optimizer="adam",loss="MSE")
rnn_model.fit(X_train, y_train, epochs=50, batch_size=256)

#check r2 scores
rnn_predictions = rnn_model.predict(X_test)
rnn_score = r2_score(y_test,rnn_predictions)
print("R2 Score of RNN model: ",rnn_score)

# Mean Forecast Error
forecast_errors = [y_test[i]-rnn_predictions[i] for i in range(len(y_test))]
bias = sum(forecast_errors) * 1.0/len(y_test)
print('Bias: %f' % bias)

# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, rnn_predictions)
print('MAE: %f' % mae)

# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y_test, rnn_predictions)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

#compare the actual values vs predicted values by plotting a graph
def plot_predictions(test, predicted, title):
    plt.figure(figsize=(16,4))
    plt.plot(test, color='blue',label='Actual power consumption data')
    plt.plot(predicted, alpha=0.7, color='orange',label='Predicted power consumption data')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Normalized power consumption scale')
    plt.legend()
    plt.show()
    
plot_predictions(y_test, rnn_predictions, "SIB1001 Predictions made by simple RNN model")

#build an LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(40,activation="tanh",return_sequences=True, input_shape=(X_train.shape[1],1)))
lstm_model.add(Dropout(0.15))
lstm_model.add(LSTM(40,activation="tanh",return_sequences=True))
lstm_model.add(Dropout(0.15))
lstm_model.add(LSTM(40,activation="tanh",return_sequences=False))
lstm_model.add(Dropout(0.15))
lstm_model.add(Dense(1))
lstm_model.summary()

#evaluate model performance
lstm_model.compile(optimizer="adam",loss="MSE")
lstm_model.fit(X_train, y_train, epochs=50, batch_size=256)

#check r2 score
lstm_predictions = lstm_model.predict(X_test)
lstm_score = r2_score(y_test, lstm_predictions)
print("R^2 Score of LSTM model: ",lstm_score)

# Forecast Mean Error
forecast_errors = [y_test[i]-lstm_predictions[i] for i in range(len(y_test))]
bias = sum(forecast_errors) * 1.0/len(y_test)
print('Bias: %f' % bias)

# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, lstm_predictions)
print('MAE: %f' % mae)

# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y_test, lstm_predictions)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

#compare the actual values vs predicted values by plotting a graph
plot_predictions(y_test, lstm_predictions, "SIB1001 Predictions made by LSTM model")

#compare predictions made by simple RNN and LSTM model by plotting data in a single graph
plt.figure(figsize=(15,8))
plt.plot(y_test, c="orange", linewidth=3, label="Original values")
plt.plot(lstm_predictions, c="red", linewidth=3, label="LSTM predictions")
plt.plot(rnn_predictions, alpha=0.5, c="green", linewidth=3, label="RNN predictions")
plt.legend()
plt.title("SIB1001 Predictions vs Actual Data", fontsize=20)
plt.show()

In [8]:
# Univariate Analysis for HAR2002

In [ ]:
#dropping rows with missing data
HAR2002DF = HAR2002DF.dropna()
#checking missing data
HAR2002DF.isna().sum()

#column data type formatting
HAR2002DF.datetime = pd.to_datetime(HAR2002DF.datetime)
HAR2002DF.HAR2002 = pd.to_numeric(HAR2002DF.HAR2002)
HAR2002DF.set_index(['datetime'], inplace = True)
HAR2002DF.head()

#visualize data before normalization
HAR2002DF.plot(figsize=(16,4),legend=True)

plt.title('Hourly power consumption data - BEFORE NORMALIZATION')

plt.show()

#normalize data
def normalize_data(HAR2002DF):
    scaler = sklearn.preprocessing.MinMaxScaler()
    HAR2002DF['HAR2002']=scaler.fit_transform(HAR2002DF['HAR2002'].values.reshape(-1,1))
    return HAR2002DF
HAR2002DF_norm = normalize_data(HAR2002DF)
HAR2002DF_norm.shape

#visualize data after normalization
HAR2002DF_norm.plot(figsize=(16,4),legend=True)
plt.title('HAR2002 hourly power consumption data - AFTER NORMALIZATION')
plt.show()

HAR2002DF_norm.shape

#prepare data for training the RNN models
def load_data(stock, seq_len):
    X_train = []
    y_train = []
    for i in range(seq_len, len(stock)):
        X_train.append(stock.iloc[i-seq_len : i, 0])
        y_train.append(stock.iloc[i, 0])
    
    #1 last 3187 days are going to be used in test
    X_test = X_train[21000:]             
    y_test = y_train[21000:]
    
    #2 first 21000 days are going to be used in training
    X_train = X_train[:21000]           
    y_train = y_train[:21000]
    
    #3 convert to numpy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    #4 reshape data to input into RNN models
    X_train = np.reshape(X_train, (21000, seq_len, 1))
    
    X_test = np.reshape(X_test, (X_test.shape[0], seq_len, 1))
    
    return [X_train, y_train, X_test, y_test]

#create train, test data
seq_len = 20 #choose sequence length
X_train, y_train, X_test, y_test = load_data(HAR2002DF, seq_len)
print('X_train.shape = ',X_train.shape)
print('y_train.shape = ', y_train.shape)
print('X_test.shape = ', X_test.shape)
print('y_test.shape = ',y_test.shape)


#build a simple RNN model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=True, input_shape=(X_train.shape[1],1)))
rnn_model.add(Dropout(0.15))
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=True))
rnn_model.add(Dropout(0.15))
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=False))
rnn_model.add(Dropout(0.15))
rnn_model.add(Dense(1))
rnn_model.summary()

#evaluate model performance
rnn_model.compile(optimizer="adam",loss="MSE")
rnn_model.fit(X_train, y_train, epochs=50, batch_size=256)

#check r2 scores
rnn_predictions = rnn_model.predict(X_test)
rnn_score = r2_score(y_test,rnn_predictions)
print("R2 Score of RNN model: ",rnn_score)

# Mean Forecast Error
forecast_errors = [y_test[i]-rnn_predictions[i] for i in range(len(y_test))]
bias = sum(forecast_errors) * 1.0/len(y_test)
print('Bias: %f' % bias)

# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, rnn_predictions)
print('MAE: %f' % mae)

# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y_test, rnn_predictions)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

#compare the actual values vs predicted values by plotting a graph
def plot_predictions(test, predicted, title):
    plt.figure(figsize=(16,4))
    plt.plot(test, color='blue',label='Actual power consumption data')
    plt.plot(predicted, alpha=0.7, color='orange',label='Predicted power consumption data')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Normalized power consumption scale')
    plt.legend()
    plt.show()
    
plot_predictions(y_test, rnn_predictions, "HAR2002 Predictions made by simple RNN model")

#build an LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(40,activation="tanh",return_sequences=True, input_shape=(X_train.shape[1],1)))
lstm_model.add(Dropout(0.15))
lstm_model.add(LSTM(40,activation="tanh",return_sequences=True))
lstm_model.add(Dropout(0.15))
lstm_model.add(LSTM(40,activation="tanh",return_sequences=False))
lstm_model.add(Dropout(0.15))
lstm_model.add(Dense(1))
lstm_model.summary()

#evaluate model performance
lstm_model.compile(optimizer="adam",loss="MSE")
lstm_model.fit(X_train, y_train, epochs=50, batch_size=256)

#check r2 score
lstm_predictions = lstm_model.predict(X_test)
lstm_score = r2_score(y_test, lstm_predictions)
print("R^2 Score of LSTM model: ",lstm_score)

# Forecast Mean Error
forecast_errors = [y_test[i]-lstm_predictions[i] for i in range(len(y_test))]
bias = sum(forecast_errors) * 1.0/len(y_test)
print('Bias: %f' % bias)

# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, lstm_predictions)
print('MAE: %f' % mae)

# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y_test, lstm_predictions)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

#compare the actual values vs predicted values by plotting a graph
plot_predictions(y_test, lstm_predictions, "HAR2002 Predictions made by LSTM model")

#compare predictions made by simple RNN and LSTM model by plotting data in a single graph
plt.figure(figsize=(15,8))
plt.plot(y_test, c="orange", linewidth=3, label="Original values")
plt.plot(lstm_predictions, c="red", linewidth=3, label="LSTM predictions")
plt.plot(rnn_predictions, alpha=0.5, c="green", linewidth=3, label="RNN predictions")
plt.legend()
plt.title("HAR2002 Predictions vs Actual Data", fontsize=20)
plt.show()

In [10]:
# Univariate Analysis for TIM2002

In [ ]:
#dropping rows with missing data
TIM2002DF = TIM2002DF.dropna()
#checking missing data
TIM2002DF.isna().sum()

#column data type formatting
TIM2002DF.datetime = pd.to_datetime(TIM2002DF.datetime)
TIM2002DF.TIM2002 = pd.to_numeric(TIM2002DF.TIM2002)
TIM2002DF.set_index(['datetime'], inplace = True)
TIM2002DF.head()

#visualize data before normalization
TIM2002DF.plot(figsize=(16,4),legend=True)

plt.title('Hourly power consumption data - BEFORE NORMALIZATION')

plt.show()

#normalize data
def normalize_data(TIM2002DF):
    scaler = sklearn.preprocessing.MinMaxScaler()
    TIM2002DF['TIM2002']=scaler.fit_transform(TIM2002DF['TIM2002'].values.reshape(-1,1))
    return TIM2002DF
TIM2002DF_norm = normalize_data(TIM2002DF)
TIM2002DF_norm.shape

#visualize data after normalization
TIM2002DF_norm.plot(figsize=(16,4),legend=True)
plt.title('TIM2002 hourly power consumption data - AFTER NORMALIZATION')
plt.show()

TIM2002DF_norm.shape

#prepare data for training the RNN models
def load_data(stock, seq_len):
    X_train = []
    y_train = []
    for i in range(seq_len, len(stock)):
        X_train.append(stock.iloc[i-seq_len : i, 0])
        y_train.append(stock.iloc[i, 0])
    
    #1 last 3187 days are going to be used in test
    X_test = X_train[21000:]             
    y_test = y_train[21000:]
    
    #2 first 21000 days are going to be used in training
    X_train = X_train[:21000]           
    y_train = y_train[:21000]
    
    #3 convert to numpy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    #4 reshape data to input into RNN models
    X_train = np.reshape(X_train, (21000, seq_len, 1))
    
    X_test = np.reshape(X_test, (X_test.shape[0], seq_len, 1))
    
    return [X_train, y_train, X_test, y_test]

#create train, test data
seq_len = 20 #choose sequence length
X_train, y_train, X_test, y_test = load_data(TIM2002DF, seq_len)
print('X_train.shape = ',X_train.shape)
print('y_train.shape = ', y_train.shape)
print('X_test.shape = ', X_test.shape)
print('y_test.shape = ',y_test.shape)
X_train.shape =  (21000, 20, 1)
y_train.shape =  (21000,)
X_test.shape =  (3797, 20, 1)
y_test.shape =  (3797,)

#build a simple RNN model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=True, input_shape=(X_train.shape[1],1)))
rnn_model.add(Dropout(0.15))
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=True))
rnn_model.add(Dropout(0.15))
rnn_model.add(SimpleRNN(40,activation="tanh",return_sequences=False))
rnn_model.add(Dropout(0.15))
rnn_model.add(Dense(1))
rnn_model.summary()

#evaluate model performance
rnn_model.compile(optimizer="adam",loss="MSE")
rnn_model.fit(X_train, y_train, epochs=50, batch_size=256)

#check r2 scores
rnn_predictions = rnn_model.predict(X_test)
rnn_score = r2_score(y_test,rnn_predictions)
print("R2 Score of RNN model: ",rnn_score)

# Mean Forecast Error
forecast_errors = [y_test[i]-rnn_predictions[i] for i in range(len(y_test))]
bias = sum(forecast_errors) * 1.0/len(y_test)
print('Bias: %f' % bias)

# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, rnn_predictions)
print('MAE: %f' % mae)

# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y_test, rnn_predictions)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

#compare the actual values vs predicted values by plotting a graph
def plot_predictions(test, predicted, title):
    plt.figure(figsize=(16,4))
    plt.plot(test, color='blue',label='Actual power consumption data')
    plt.plot(predicted, alpha=0.7, color='orange',label='Predicted power consumption data')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Normalized power consumption scale')
    plt.legend()
    plt.show()
    
plot_predictions(y_test, rnn_predictions, "TIM2002 Predictions made by simple RNN model")

#build an LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(40,activation="tanh",return_sequences=True, input_shape=(X_train.shape[1],1)))
lstm_model.add(Dropout(0.15))
lstm_model.add(LSTM(40,activation="tanh",return_sequences=True))
lstm_model.add(Dropout(0.15))
lstm_model.add(LSTM(40,activation="tanh",return_sequences=False))
lstm_model.add(Dropout(0.15))
lstm_model.add(Dense(1))
lstm_model.summary()

#evaluate model performance
lstm_model.compile(optimizer="adam",loss="MSE")
lstm_model.fit(X_train, y_train, epochs=50, batch_size=256)

#check r2 score
lstm_predictions = lstm_model.predict(X_test)
lstm_score = r2_score(y_test, lstm_predictions)
print("R^2 Score of LSTM model: ",lstm_score)

# Forecast Mean Error
forecast_errors = [y_test[i]-lstm_predictions[i] for i in range(len(y_test))]
bias = sum(forecast_errors) * 1.0/len(y_test)
print('Bias: %f' % bias)

# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, lstm_predictions)
print('MAE: %f' % mae)

# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y_test, lstm_predictions)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

#compare the actual values vs predicted values by plotting a graph
plot_predictions(y_test, lstm_predictions, "TIM2002 Predictions made by LSTM model")

#compare predictions made by simple RNN and LSTM model by plotting data in a single graph
plt.figure(figsize=(15,8))
plt.plot(y_test, c="orange", linewidth=3, label="Original values")
plt.plot(lstm_predictions, c="red", linewidth=3, label="LSTM predictions")
plt.plot(rnn_predictions, alpha=0.5, c="green", linewidth=3, label="RNN predictions")
plt.legend()
plt.title("TIM2002 Predictions vs Actual Data", fontsize=20)
plt.show()